In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [42]:
data = pd.read_csv("/kaggle/input/youtube-new/USvideos.csv")
data.head()

In [43]:
data.info()

In [45]:
data.drop("thumbnail_link", axis = 1, inplace = True)
data.head()

In [46]:
data.drop("video_id", axis = 1, inplace = True)
data.head()

In [47]:
data.drop("category_id", axis = 1, inplace = True)
data.head()

In [49]:
data["comments_disabled"] = data["comments_disabled"].apply(lambda x: int(x))

In [53]:
data["ratings_disabled"] = data["ratings_disabled"].astype(int)

In [52]:
data["video_error_or_removed"].replace({False:0, True:1}, inplace=True)
data.head()

In [54]:
data["publish_date"] = data["publish_time"].apply(lambda x: x.split('T')[0])
data["publish_hour_mins"] = data["publish_time"].apply(lambda x: x.split('T')[1][0:5])
data["publish_mins"] = data["publish_hour_mins"].apply(lambda x:float(x.split(':')[0])*60+float(x.split(':')[1]))
data.head()

In [56]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
# np.array([2.4, 6.2, 1.8, 9.0]).reshape(-1, 1).unique()
data["channel_title"] = encoder.fit_transform(np.array(data["channel_title"]).reshape(-1,1))

In [57]:
import seaborn as sns

sns.set_palette("GnBu_d")
sns.set_style("whitegrid")

# X = data[["channel_title",  "publish_mins", "views", "likes", "dislikes", "publish_date"]]
# sns.pairplot(X)

In [58]:
sns.distplot(data["views"])

In [59]:
data["views"].hist(bins=100)

In [82]:
data["label"] = -1
    
data.loc[(data["views"] > 2000000), "label"] = 5
data.loc[((data["views"] > 1000000) & (data["views"] <= 2000000)), "label"] = 4
data.loc[((data["views"] > 500000) & (data["views"] <= 1000000)), "label"] = 3
data.loc[((data["views"] > 250000) & (data["views"] <= 500000)), "label"] = 2
data.loc[data["views"] <= 250000, "label"] = 1
# data.loc[(data["views"] <= 100000), "label"] = 1

data["label"].value_counts()

In [ ]:
data["month"] = data["trending_date"].apply(lambda x: int(x.split('.')[1]))
data["year"] = data["trending_date"].apply(lambda x: int(x.split('.')[0]))
data.head()
data["year"].value_counts()

In [ ]:
# X2 = data[["channel_title", "publish_mins", "year", "month", "trending_date"]]
# sns.pairplot(X2)


In [ ]:
# import string
# string.punctuation

# def removePunc(text):
#     return "".join([char for char in text if char not in string.punctuation])

# data["noPunc"] = data["titleTags"].apply(lambda x: removePunc(x))
# data.head()

In [113]:
data["titleTags"] = data["title"]+" "+data["tags"]
data["titleTags"] = data["titleTags"].apply(lambda x: x.lower())

In [ ]:
# import re
# import nltk
# data["noPuncTokens"] = data["noPunc"].apply(lambda x: re.split("\W+", x.lower()))

# # f = "/Users/adkulkar/Downloads/stopwords/english"
# # with open(f, 'r') as file:
# #     stopwords = file.read().replace('\n', '')
# # stopwords = pd.read_csv("/usr/local/share/nltk_data/corpora/stopwords/english")

# def removeStopWords(text):
#     textNoStops = [word for word in text if word not in nltk.corpus.stopwords.words("english")]
#     return textNoStops

# data["noStopWords"] = data["noPuncTokens"].apply(lambda x: removeStopWords(x))

# data.head()



In [ ]:
# nltk.download("wordnet")
# from nltk.stem import WordNetLemmatizer

# wn = WordNetLemmatizer()

# def lemmatizer(text):
#     return [wn.lemmatize(x) for x in text]

# data['lemmatizedTokens'] = data["noStopWords"].apply(lambda x: lemmatizer(x))
# data.head()

In [114]:
import nltk
# nltk.download("wordnet")
from nltk.stem import WordNetLemmatizer
import string
string.punctuation

wn = WordNetLemmatizer()

def text_process(text):
    nopunc = [c for c in text if c not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [wn.lemmatize(word) for word in nopunc.split() if word not in nltk.corpus.stopwords.words("english")]


In [ ]:
# from sklearn.pipeline import Pipeline
# from sklearn.feature_extraction.text import CountVectorizer
# from sklearn.feature_extraction.text import TfidfTransformer
# from sklearn.naive_bayes import MultinomialNB

# pipeline = Pipeline([
#     ('bow', CountVectorizer(analyzer=text_process)),
#     ('tfidf', TfidfTransformer()),
#     ('classifier', MultinomialNB())
# ])

In [ ]:
# pipeline.fit(xtrain, ytrain)

In [115]:
from sklearn.feature_extraction.text import CountVectorizer

bow_transformer = CountVectorizer(analyzer= text_process).fit(data["titleTags"])
messages_bow = bow_transformer.transform(data["titleTags"])

In [117]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf = tfidf_transformer.transform(messages_bow)

In [120]:
tfidf.shape

In [93]:
data["publish_date"]

In [122]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(tfidf, data['label'], test_size = 0.4, random_state = 101)

In [136]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth = 25, random_state = 0)
model.fit(xtrain, ytrain)

predictions = model.predict(xtest)

In [137]:
from sklearn.metrics import classification_report 

print(classification_report(ytest, predictions))